# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json 


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = pd.read_csv("../output_data/cities.csv")

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [13]:
output_data_file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


## Generate Cities List

In [14]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


626

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [9]:
cities[:8]

['airai',
 'mwinilunga',
 'klaksvik',
 'takoradi',
 'albany',
 'yicheng',
 'oksfjord',
 'punta arenas']

In [ ]:
#create empty lists
city_name = []
cloudiness = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []

#weather api 
url = 'http://api.openweathermap.org/data/2.5/weather'

#print statement
print('Beginning Data Retrieval')
print('-----------------------------')

record = 0
set_count = 1
for city in cities:
    #create parameters for api call 
    params={
        "q":city,
        "units":"imperial",
        "appid": weather_api_key
    }
    try:
        response = requests.get(url, params)
        #see if API call successful
        #print(response.url)
        results = response.json()
        #reformat to see data better
        #print(json.dumps(results, indent=4, sort_keys=True))
        cloudiness.append(results["clouds"]["all"])
        date.append(results["dt"])
        humidity.append(results["main"]["humidity"])
        lat.append(results["coord"]["lat"])
        lng.append(results["coord"]["lon"])
        max_temp.append(results["main"]["temp_max"])
        wind_speed.append(results["wind"]["speed"])
        #icrease counter by one 
        record = record+1
        print(f"Processing Record{record}|{city}")
    
    except Exception:
        print("City not found. Skipping...")

print("------------------------------")
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record1|saint anthony
Processing Record2|luderitz
Processing Record3|kodiak
Processing Record4|avarua
Processing Record5|bengkulu
Processing Record6|nisia floresta
Processing Record7|hermanus
Processing Record8|lasa
Processing Record9|jamestown
Processing Record10|turayf
Processing Record11|yellowknife
Processing Record12|kapaa
Processing Record13|independence
Processing Record14|albany
Processing Record15|okitipupa
Processing Record16|denpasar
Processing Record17|nikolskoye
Processing Record18|atuona
Processing Record19|busselton
Processing Record20|opuwo
Processing Record21|bredasdorp
Processing Record22|yar-sale
Processing Record23|tilichiki
Processing Record24|hinton
Processing Record25|mataura
Processing Record26|barrow
Processing Record27|naze
Processing Record28|rosario
Processing Record29|rikitea
Processing Record30|orapa
Processing Record31|mana
Processing Record32|saint-philippe
Processing Record33|puerto ayora

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot

## Linear Regression

In [6]:
# OPTIONAL: Create a function to create Linear Regression plots


In [7]:
# Create Northern and Southern Hemisphere DataFrames


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression